In [1]:
!pip install pymongo

In [1]:
from pymongo import MongoClient

# Inisialisasi client dan koneksi ke database
client = MongoClient('mongodb://localhost:27017/')
db = client['company_db']
collection = db['employees']

# Contoh untuk memastikan koneksi
print('Connected to MongoDB')

Connected to MongoDB


In [2]:
# Diasumsikan variabel 'db' sudah ada dari koneksi database sebelumnya

# 1. Membuat atau memilih koleksi 'products'
products_collection = db['products']

# 2. Menyiapkan daftar data produk (lebih dari 10)
# Kita buat data yang bervariasi agar query nanti lebih menarik
product_list = [
    {'name': 'MacBook Pro 15', 'category': 'Electronics', 'price': 1500, 'stock': 50},
    {'name': 'Smartphone Xiaomi', 'category': 'Electronics', 'price': 800, 'stock': 200},
    {'name': 'Wireless Mouse', 'category': 'Accessories', 'price': 50, 'stock': 500},
    {'name': 'Mechanical Keyboard', 'category': 'Accessories', 'price': 120, 'stock': 300},
    {'name': '4K Monitor 27inch"', 'category': 'Electronics', 'price': 450, 'stock': 100},
    {'name': 'Python for Beginners', 'category': 'Books', 'price': 40, 'stock': 1000},
    {'name': 'MongoDB in Action', 'category': 'Books', 'price': 55, 'stock': 800},
    {'name': 'USB-C Hub', 'category': 'Accessories', 'price': 75, 'stock': 400},
    {'name': 'Webcam HD1080', 'category': 'Electronics', 'price': 90, 'stock': 250},
    {'name': 'The Pragmatic Programmer', 'category': 'Books', 'price': 60, 'stock': 750},
    {'name': 'TUF Gaming Laptop A17', 'category': 'Electronics', 'price': 2100, 'stock': 30}
]

# 3. Memasukkan semua data ke dalam koleksi 'products'
# Hapus dulu data lama (jika ada) agar tidak duplikat saat kode dijalankan ulang
products_collection.delete_many({}) 
result = products_collection.insert_many(product_list)

print(f"Berhasil membuat koleksi 'products' dan menambahkan {len(result.inserted_ids)} data produk.")

Berhasil membuat koleksi 'products' dan menambahkan 11 data produk.


In [3]:
# --- LANGKAH 1: Hitung harga rata-rata ---
# Buat pipeline untuk mengelompokkan SEMUA dokumen dan menghitung rata-rata harganya.
# _id: null berarti kita mengelompokkan semua dokumen menjadi satu grup.
avg_pipeline = [
    {'$group': {'_id': None, 'average_price': { '$avg': '$price' }}}
]

# Jalankan agregasi. Hasilnya adalah sebuah list berisi satu dokumen.
avg_result = list(products_collection.aggregate(avg_pipeline))

# Cek jika ada hasil, lalu ambil nilainya
if avg_result:
    avg_price = avg_result[0]['average_price']
    print(f"Harga rata-rata semua produk adalah: ${avg_price:.2f}")

    # --- LANGKAH 2: Cari produk dengan harga di atas rata-rata ---
    print("\nProduk dengan harga di atas rata-rata:")
    
    # Gunakan operator $gt (greater than) untuk memfilter harga
    query_filter = { 'price': { '$gt': avg_price } }
    
    products_above_avg = products_collection.find(query_filter)
    
    for product in products_above_avg:
        print(f"- {product['name']} (Harga: ${product['price']})")
else:
    print("Tidak ada produk untuk dihitung rata-ratanya.")

Harga rata-rata semua produk adalah: $485.45

Produk dengan harga di atas rata-rata:
- MacBook Pro 15 (Harga: $1500)
- Smartphone Xiaomi (Harga: $800)
- TUF Gaming Laptop A17 (Harga: $2100)


In [4]:
# Buat pipeline untuk mengelompokkan berdasarkan kategori dan menghitung jumlahnya
count_by_category_pipeline = [
    {'$group': {'_id': '$category','jumlah_produk': { '$sum': 1 }}},
    {'$sort': { '_id': 1 }}
]

print("\n--- Jumlah Total Produk per Kategori ---")
# Jalankan agregasi dan cetak hasilnya
for category_data in products_collection.aggregate(count_by_category_pipeline):
    # category_data['_id'] akan berisi nama kategori
    # category_data['jumlah_produk'] akan berisi hasil hitungannya
    print(f"Kategori: {category_data['_id']}, Jumlah Produk: {category_data['jumlah_produk']}")


--- Jumlah Total Produk per Kategori ---
Kategori: Accessories, Jumlah Produk: 3
Kategori: Books, Jumlah Produk: 3
Kategori: Electronics, Jumlah Produk: 5
